In [1]:
import pandas as pd
import json, requests
from mapboxgl.utils import df_to_geojson
import sys
from IPython.display import display
from branca.colormap import linear

In [2]:
import pandas as pd
path = '../data/LOCAL_PEOPLE_GU_2019.csv'

df = pd.read_csv(path, encoding='cp949',index_col = False)
df

C:\Users\jason\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


,기준일ID,시간대구분,자치구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20190101,0,11110,247783.9476,6613.1044,3564.9470,6134.7190,12757.6730,13424.9912,9679.4043,...,11592.6358,8455.5240,8669.4651,7764.7926,9659.7779,8421.0455,8671.2045,8079.0106,6242.6970,17962.9680
1,20190101,0,11140,176516.2769,4735.9065,1857.9963,3251.4707,7456.5552,8482.9745,8253.8513,...,8654.7543,7425.3943,7345.0904,5858.8981,6255.6065,5525.2377,6293.4810,5906.2118,4298.7062,12303.3213
2,20190101,0,11170,287806.8492,7274.7405,3654.3768,5865.4531,11656.1407,15548.7770,14457.9881,...,16203.3581,13423.3780,13009.2350,10794.0704,10880.1619,8360.3558,9568.1641,8703.7465,6648.3724,20296.2431
3,20190101,0,11200,335106.5204,12278.8839,5306.1528,6766.7787,11593.4668,13941.8426,13447.0673,...,13656.2699,14046.9836,15142.1260,12029.9710,13051.1879,11475.2065,13219.4730,12022.8741,9093.4057,24370.0786
4,20190101,0,11215,399818.0269,12003.9916,6472.3827,12056.0837,17503.8179,18611.2942,16219.1953,...,17737.9096,16189.7277,16158.7700,14059.8023,15750.9307,14705.5715,15708.3156,14021.6731,9907.6266,24975.8391
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211195,20191231,23,11620,512045.7410,13292.1313,6492.1774,10001.0660,20423.9464,34108.8311,27984.1499,...,29361.7592,22645.1803,20459.1792,16839.3671,17200.5078,16012.9977,18377.4578,16540.6608,13771.6139,32595.2666
211196,20191231,23,11650,507017.9562,17249.4614,9728.3280,15440.6841,15643.9679,18565.4597,17723.9030,...,18985.1085,18778.2261,23145.8814,21481.7859,22961.2336,17494.8242,17660.0288,16583.0255,12881.3183,30976.3635
211197,20191231,23,11680,701730.1925,21768.6379,14313.7773,22627.7125,21085.1486,29039.6792,28890.8864,...,31572.3410,29317.5029,32070.2938,30395.5776,31006.2854,21940.4235,22145.4239,21115.0631,16258.7322,39325.7057
211198,20191231,23,11710,776809.6518,29303.9813,15154.6352,21907.8004,21864.3757,29286.5636,29977.2304,...,33346.5810,32816.1799,35769.2569,30872.0662,31988.1565,27188.8780,31098.6350,27865.7032,20081.3478,44078.4656


In [3]:
path = '../data/LOCAL_PEOPLE_GU_2019.csv'

df = pd.read_csv(path, encoding='cp949',index_col = False)
a = df.iloc[:,7:15]
b = df.iloc[:,22:30]

df['sum'] = 0

for e in set(a.columns)|set(b.columns):
    df['sum'] += df[e]
    
c = pd.concat([df.iloc[:,0:3], df['sum']], axis=1)
df = c.groupby(['자치구코드', '시간대구분']).mean().reset_index().drop(columns=['기준일ID'])

for e in set(df['자치구코드']):
    idx = df[df['자치구코드']==e].index
    if e == 11110:
        df.loc[idx, '자치구코드'] = '종로구'
    elif e == 11140:
        df.loc[idx, '자치구코드'] = '중구'
    elif e == 11170:
        df.loc[idx, '자치구코드'] = '용산구'
    elif e == 11200:
        df.loc[idx, '자치구코드'] = '성동구'
    elif e == 11215:
        df.loc[idx, '자치구코드'] = '광진구'
    elif e == 11230:
        df.loc[idx, '자치구코드'] = '동대문구'
    elif e == 11260:
        df.loc[idx, '자치구코드'] = '중랑구'
    elif e == 11290:
        df.loc[idx, '자치구코드'] = '성북구'
    elif e == 11305:
        df.loc[idx, '자치구코드'] = '강북구'
    elif e == 11320:
        df.loc[idx, '자치구코드'] = '도봉구'
    elif e == 11350:
        df.loc[idx, '자치구코드'] = '노원구'
    elif e == 11380:
        df.loc[idx, '자치구코드'] = '은평구'
    elif e == 11410:
        df.loc[idx, '자치구코드'] = '서대문구'
    elif e == 11440:
        df.loc[idx, '자치구코드'] = '마포구'
    elif e == 11470:
        df.loc[idx, '자치구코드'] = '양천구'
    elif e == 11500:
        df.loc[idx, '자치구코드'] = '강서구'
    elif e == 11530:
        df.loc[idx, '자치구코드'] = '구로구'
    elif e == 11545:
        df.loc[idx, '자치구코드'] = '금천구'
    elif e == 11560:
        df.loc[idx, '자치구코드'] = '영등포구'
    elif e == 11590:
        df.loc[idx, '자치구코드'] = '동작구'
    elif e == 11620:
        df.loc[idx, '자치구코드'] = '관악구'
    elif e == 11650:
        df.loc[idx, '자치구코드'] = '서초구'
    elif e == 11680:
        df.loc[idx, '자치구코드'] = '강남구'
    elif e == 11710:
        df.loc[idx, '자치구코드'] = '송파구'
    elif e == 11740:
        df.loc[idx, '자치구코드'] = '강동구'
        
df = df.set_index(['자치구코드', '시간대구분']).unstack()

a = df.iloc[:,9:18]
b = df.drop(columns=df.iloc[:,9:18].columns)

df['noon'] = 0
df['night'] = 0

for e in a.columns:
    df['noon'] += df[e]
    df.drop(columns=[e], inplace=True)
for e in b.columns:
    df['night'] += df[e]
    df.drop(columns=[e], inplace=True)
df['noon'] = df['noon']/9
df['night'] = df['night']/15
df.reset_index(inplace=True)

geo_json = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'
json_data = json.loads(requests.get(geo_json).text)

json_data

C:\Users\jason\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return func(*args, **kwargs)


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.11879551821994, 37.557222485451305],
      [127.12146867175024, 37.55986003393365],
      [127.12435254630417, 37.56144246249796],
      [127.13593925898998, 37.56564793048277],
      [127.14930548011061, 37.56892250303897],
      [127.15511020940411, 37.57093642128295],
      [127.16683184366129, 37.57672487388627],
      [127.17038810813094, 37.576465605301046],
      [127.17607118428914, 37.57678573961056],
      [127.17905504160184, 37.57791388161732],
      [127.17747787800164, 37.57448983055031],
      [127.1781775408844, 37.571481967974336],
      [127.17995281860672, 37.569309661290504],
      [127.18122821955262, 37.56636089217979],
      [127.18169407550688, 37.56286338914073],
      [127.18408792

In [4]:
for e in json_data['features']:
    e['geometry']['coordinates'] = [list(map(lambda x : [x[0]-0.00250000000000, x[1] + 0.00250000000000], e['geometry']['coordinates'][0]))]
    del e['properties']['base_year']
json_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250', 'name': '강동구', 'name_eng': 'Gangdong-gu'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11269584981606, 37.56003318070491],
      [127.11629551821994, 37.5597224854513],
      [127.11896867175024, 37.56236003393365],
      [127.12185254630417, 37.563942462497955],
      [127.13343925898998, 37.568147930482766],
      [127.14680548011061, 37.57142250303897],
      [127.15261020940412, 37.57343642128295],
      [127.16433184366129, 37.57922487388627],
      [127.16788810813094, 37.57896560530104],
      [127.17357118428914, 37.579285739610555],
      [127.17655504160184, 37.58041388161732],
      [127.17497787800164, 37.57698983055031],
      [127.17567754088441, 37.57398196797433],
      [127.17745281860672, 37.5718096612905],
      [127.17872821955262, 37.568860892179785],
      [127.17919407550689, 37.56536338914073],
      [127.18158792330152, 37.56064280369575],
      [

In [5]:
d = pd.read_csv('../data/electric_charger.csv')
d=d.fillna('')
df2=d[['statNm', 'addr','lat','lng']]
df2=df2.rename(columns={'lat':'lat','lng':'lon'})
df2=df2[df2['lat']!='']
df2=df2[df2['lon']!='']
df2['lat']=df2['lat'].astype(float)
df2['lon']=df2['lon'].astype(float)

geo_data = df_to_geojson(
    df=df2,
    properties=['statNm', 'addr'],
    lat='lat', lon='lon',
    precision=5,
    filename = "parks.geojson"
)
geo_data_2 = 'parks.geojson'

with open(geo_data_2) as f:
    data = json.loads(f.read())

lst = []
dict_ = {}

for e in data['features']:
    try:
        if e['properties']['addr'].split(' ')[1] == '입구좌측,':
            e['properties']['gu'] = '관악구'
        else:
            e['properties']['gu'] = e['properties']['addr'].split(' ')[1]
            lst.append(e['properties']['gu'])
    except:
        e['properties']['gu'] = '금천구'
        lst.append('금천구')
lst = set(lst)

for e in lst:
    dict_[e] = 0
    
for e in data['features']:
    dict_[e['properties']['gu']] += 1
    

In [17]:
def percentile_cal(json_data):
    max_noon = 0
    min_noon = sys.maxsize
    max_night = 0
    min_night = sys.maxsize
    
    for e in json_data['features']:
        for e2 in list(dict_.keys()):
            if e['properties']['name'] == e2:
                #e['properties']['stations'] = dict_[e2]
                e['properties']['noon_percentile'] = e['properties']['noon']/e['properties']['stations']
                e['properties']['night_percentile'] = e['properties']['night']/e['properties']['stations']
                if e['properties']['noon_percentile'] > max_noon :
                    max_noon = e['properties']['noon_percentile']
                if e['properties']['noon_percentile'] < min_noon:
                    min_noon = e['properties']['noon_percentile']
                if e['properties']['night_percentile'] > max_night :
                    max_night = e['properties']['night_percentile']
                if e['properties']['night_percentile'] < min_night:
                    min_night = e['properties']['night_percentile']
                txt = '<b><h4>{}</h4></b>충전소 수(대): {}<br>충전소 1대 당 감당 인구(낮): {:.2f}<br>충전소 1대 당 감당 인구(밤): {:.2f}'.format(e2,e['properties']['stations'], e['properties']['noon_percentile'],e['properties']['night_percentile'])
                e['properties']['tooltip1'] = txt
    return json_data

In [18]:
percentile_cal(json_data)

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'noon': 260045.98669021463,
    'night': 295823.71497829544,
    'stations': 406.0,
    'noon_percentile': 640.5073563798389,
    'night_percentile': 728.6298398480183,
    'tooltip1': '<b><h4>강동구</h4></b>충전소 수(대): 285<br>충전소 1대 당 감당 인구(낮): 640.51<br>충전소 1대 당 감당 인구(밤): 728.63',
    'install': 64.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11269584981606, 37.56003318070491],
      [127.11629551821994, 37.5597224854513],
      [127.11896867175024, 37.56236003393365],
      [127.12185254630417, 37.563942462497955],
      [127.13343925898998, 37.568147930482766],
      [127.14680548011061, 37.57142250303897],
      [127.15261020940412, 37.57343642128295],
      [127.16433184366129, 37.57922487388627],
      [127.16788810813094, 37.57896560530104],
      [127.17357118428914, 37.579285739610555],
      [127.1765550416018

In [8]:
for e in json_data['features']:
    for e2 in df['자치구코드']:
        if e['properties']['name'] == e2:
            e['properties']['noon'] = df[df['자치구코드'] == e2]['noon'].iloc[0]
            e['properties']['night'] = df[df['자치구코드'] == e2]['night'].iloc[0]

max_noon = 0
min_noon = sys.maxsize
max_night = 0
min_night = sys.maxsize

for e in json_data['features']:
    for e2 in list(dict_.keys()):
        if e['properties']['name'] == e2:
            e['properties']['stations'] = dict_[e2]
            e['properties']['noon_percentile'] = e['properties']['noon']/e['properties']['stations']
            e['properties']['night_percentile'] = e['properties']['night']/e['properties']['stations']
            if e['properties']['noon_percentile'] > max_noon :
                max_noon = e['properties']['noon_percentile']
            if e['properties']['noon_percentile'] < min_noon:
                min_noon = e['properties']['noon_percentile']
            if e['properties']['night_percentile'] > max_night :
                max_night = e['properties']['night_percentile']
            if e['properties']['night_percentile'] < min_night:
                min_night = e['properties']['night_percentile']
            txt = '<b><h4>{}</h4></b>충전소 수(대): {}<br>충전소 1대 당 감당 인구(낮): {:.2f}<br>충전소 1대 당 감당 인구(밤): {:.2f}'.format(e2,e['properties']['stations'], e['properties']['noon_percentile'],e['properties']['night_percentile'])
            e['properties']['tooltip1'] = txt

json_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'noon': 260045.98669021463,
    'night': 295823.71497829544,
    'stations': 285,
    'noon_percentile': 912.4420585621566,
    'night_percentile': 1037.9779472922646,
    'tooltip1': '<b><h4>강동구</h4></b>충전소 수(대): 285<br>충전소 1대 당 감당 인구(낮): 912.44<br>충전소 1대 당 감당 인구(밤): 1037.98'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11269584981606, 37.56003318070491],
      [127.11629551821994, 37.5597224854513],
      [127.11896867175024, 37.56236003393365],
      [127.12185254630417, 37.563942462497955],
      [127.13343925898998, 37.568147930482766],
      [127.14680548011061, 37.57142250303897],
      [127.15261020940412, 37.57343642128295],
      [127.16433184366129, 37.57922487388627],
      [127.16788810813094, 37.57896560530104],
      [127.17357118428914, 37.579285739610555],
      [127.17655504160184, 37.58041388161732]

In [9]:
def make_data(json_data):
    df = pd.DataFrame([])
    idx = 0
    for e in json_data['features']:
        
        df.loc[idx, 'name'] = e['properties']['name']
        df.loc[idx, 'name_eng'] = e['properties']['name_eng']
        df.loc[idx, 'noon'] = e['properties']['noon']
        df.loc[idx, 'night'] = e['properties']['night']
        df.loc[idx, 'stations'] = e['properties']['stations']
        df.loc[idx, 'noon_percentile'] = e['properties']['noon_percentile']
        df.loc[idx, 'night_percentile'] = e['properties']['night_percentile']
        try:
            df.loc[idx, 'install'] = e['properties']['install']
        except:
            pass
        idx += 1

    return df

In [10]:
def install(year, data, json_data):
    for e1 in json_data['features']:
        for e2 in data[year]:
            if e1['properties']['name'] == e2:
                e1['properties']['install'] = data[year][e2]
                e1['properties']['stations'] += data[year][e2]
    return json_data

In [11]:
make_data(json_data)

,name,name_eng,noon,night,stations,noon_percentile,night_percentile
0,강동구,Gangdong-gu,260045.986690,295823.714978,285.0,912.442059,1037.977947
1,송파구,Songpa-gu,469660.421729,471872.300279,540.0,869.741522,873.837593
2,강남구,Gangnam-gu,664934.519079,465326.925569,660.0,1007.476544,705.040796
3,서초구,Seocho-gu,450708.846949,342209.423613,751.0,600.144936,455.671669
4,관악구,Gwanak-gu,278161.474542,336942.093492,254.0,1095.123916,1326.543675
5,동작구,Dongjak-gu,223512.352787,257178.359332,307.0,728.053266,837.714526
6,영등포구,Yeongdeungpo-gu,359691.293567,288125.145915,421.0,854.373619,684.382769
7,금천구,Geumcheon-gu,165505.493276,139008.257913,365.0,453.439708,380.844542
8,구로구,Guro-gu,245558.027474,247917.425536,495.0,496.076823,500.843284
9,강서구,Gangseo-gu,315460.667253,343064.182929,609.0,517.997812,563.323781


In [12]:
path = "../data/Add.csv"

df = pd.read_csv(path).set_index('Unnamed: 0')
df.to_json("../data/practice.json",orient='columns')

geo_data_2 = '../data/practice.json'

with open(geo_data_2) as f:
    data = json.loads(f.read())
data

{'Year1': {'강동구': 0.0,
  '송파구': 0.0,
  '강남구': 192.0,
  '서초구': 0.0,
  '관악구': 0.0,
  '동작구': 0.0,
  '영등포구': 0.0,
  '금천구': 0.0,
  '구로구': 0.0,
  '강서구': 0.0,
  '양천구': 0.0,
  '마포구': 0.0,
  '서대문구': 0.0,
  '은평구': 0.0,
  '노원구': 0.0,
  '도봉구': 0.0,
  '강북구': 0.0,
  '성북구': 0.0,
  '중랑구': 0.0,
  '동대문구': 0.0,
  '광진구': 0.0,
  '성동구': 0.0,
  '용산구': 0.0,
  '중구': 0.0,
  '종로구': 0.0},
 'Year2': {'강동구': 0.0,
  '송파구': 141.0,
  '강남구': 51.0,
  '서초구': 0.0,
  '관악구': 0.0,
  '동작구': 0.0,
  '영등포구': 0.0,
  '금천구': 0.0,
  '구로구': 0.0,
  '강서구': 0.0,
  '양천구': 0.0,
  '마포구': 0.0,
  '서대문구': 0.0,
  '은평구': 0.0,
  '노원구': 0.0,
  '도봉구': 0.0,
  '강북구': 0.0,
  '성북구': 0.0,
  '중랑구': 0.0,
  '동대문구': 0.0,
  '광진구': 0.0,
  '성동구': 0.0,
  '용산구': 0.0,
  '중구': 0.0,
  '종로구': 0.0},
 'Year3': {'강동구': 0.0,
  '송파구': 24.0,
  '강남구': 0.0,
  '서초구': 0.0,
  '관악구': 64.0,
  '동작구': 0.0,
  '영등포구': 84.0,
  '금천구': 0.0,
  '구로구': 0.0,
  '강서구': 0.0,
  '양천구': 0.0,
  '마포구': 20.0,
  '서대문구': 0.0,
  '은평구': 0.0,
  '노원구': 0.0,
  '도봉구': 0.0,
  '강북구': 0.0,
  '성북구': 0.0,
  '중

In [29]:
import folium

center = [37.541, 126.986]

def mapper(json_data, i):
    
    m_noon = folium.Map(location=center, zoom_start=11)
    df = make_data(json_data)
    choropleth_noon = folium.Choropleth(
        geo_data = json_data,
        data = df,
        columns=('name', 'noon_percentile'),
        key_on = 'feature.properties.name',
        fill_color = "YlGn",
        center = [126.986, 37.565],
        zoom=10,
        bins = [0, 300, 600, 900, 1200, 1500]
    ).add_to(m_noon)
    
    choropleth_noon.geojson.add_child(folium.features.GeoJsonTooltip(['tooltip1'], labels=False))
    title_html = '<h3 align="center" style="font-size:20px"><b>{}년 충전소 1대 당 감당 인구 수(낮)</b></h3>'.format(i+2021)
    m_noon.get_root().html.add_child(folium.Element(title_html))
    folium.LayerControl().add_to(m_noon)
    

    m_night = folium.Map(location=center, zoom_start=11)
    choropleth_night = folium.Choropleth(
        geo_data = json_data,
        data = make_data(json_data),
        columns=('name','night_percentile'),
        key_on = 'feature.properties.name',
        fill_color = "BuPu",
        zoom=10,
        bins = [0, 300, 600, 900, 1200, 1500]
    ).add_to(m_night)
    

    
    choropleth_night.geojson.add_child(folium.features.GeoJsonTooltip(['tooltip1'], labels=False))
    title_html = '<h3 align="center" style="font-size:20px"><b>{}년 충전소 1대 당 감당 인구 수(밤)</b></h3>'.format(i+2021)
    m_night.get_root().html.add_child(folium.Element(title_html))
    folium.LayerControl().add_to(m_night)
    
    display(m_noon)
    display(m_night)
    return m_noon, m_night

In [15]:
json_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'noon': 260045.98669021463,
    'night': 295823.71497829544,
    'stations': 406.0,
    'noon_percentile': 640.5073563798389,
    'night_percentile': 728.6298398480183,
    'tooltip1': '<b><h4>서초구</h4></b>충전소 수(대): 751<br>충전소 1대 당 감당 인구(낮): 640.51<br>충전소 1대 당 감당 인구(밤): 728.63',
    'install': 64.0},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11269584981606, 37.56003318070491],
      [127.11629551821994, 37.5597224854513],
      [127.11896867175024, 37.56236003393365],
      [127.12185254630417, 37.563942462497955],
      [127.13343925898998, 37.568147930482766],
      [127.14680548011061, 37.57142250303897],
      [127.15261020940412, 37.57343642128295],
      [127.16433184366129, 37.57922487388627],
      [127.16788810813094, 37.57896560530104],
      [127.17357118428914, 37.579285739610555],
      [127.1765550416018

In [30]:
init_noon, init_night = mapper(json_data, 0)
init_noon.save('./noon_0.html')
init_night.save('./night_0.html')

for i in range(1, 6):
    json_data = install('Year{}'.format(i), data, json_data)
    json_data = percentile_cal(json_data)
    
    noon, night = mapper(json_data, i)
    noon.save('./noon_{}.html')
    night.save('./night_{}.html')
    
    df_0 = make_data(json_data)
    df_0.to_csv('./Year{}.csv'.format(i), encoding='cp949')